In [ ]:
import glob
import os
import shutil

from astropy import stats
from astropy.io import fits
import numpy as np
import pandas as pd

In [ ]:
working_directory = '/grp/hst/wfc3u/final_postflash_2021'
database_filename = working_directory + '/Feb_2022_all_postflash_flc.pkl'
updated_filename = working_directory + 'Feb_2022_flc_all_stats_postflash.pkl'

In [ ]:
postflash_data = pd.read_pickle(database_filename)

In [ ]:
postflash_data[0:10]

In [ ]:
subarray_pf = postflash_data.loc[(postflash_data['subarray'] == True)] 
fullframe_pf = postflash_data.loc[(postflash_data['subarray'] == False)] 
sub_paths = subarray_pf.path.tolist()
all_paths = postflash_data.path.tolist()
print (all_paths)

In [ ]:
means = []
sig_means = []
medians = []
sig_medians = []
stds = []
sig_stds = []

for f in all_paths:
    path = f
    print(path)
    data = fits.getdata(path)
    data_clip = stats.sigma_clip(data, 3)
    
    mean = np.mean(data)
    sig_mean = np.mean(data_clip)
    median = np.median(data)
    sig_median = np.median(data_clip)
    std = np.std(data)
    sig_std = np.std(data_clip)
    
    
    means.append(mean)
    sig_means.append(sig_mean)
    medians.append(median)
    sig_medians.append(sig_median)
    stds.append(std)
    sig_stds.append(sig_std)


In [ ]:
dict = {'path': all_paths, 'mean': means, 'sig_mean': sig_means,'median': medians, 'sig_median': sig_medians, 'std': stds, 'sig_std': sig_stds}
path_and_stats_sub = pd.DataFrame(dict)

all_stats = pd.merge(left=postflash_data, right=path_and_stats_sub, left_on='path', right_on='path').set_index(postflash_data.index)

In [ ]:
all_stats['norm_by_dur'] = (all_stats['sig_mean']/all_stats['flash_dur'])
all_stats.to_pickle(updated_filename)
all_stats = pd.read_pickle(updated_filename)
print (all_stats)

In [ ]:
 all_stats['datetime'] = pd.to_datetime(all_stats['obsdate'] + ' ' + all_stats['obstime'])
 all_stats.to_pickle(updated_filename)